In [ ]:
import numpy as np
import statsmodels.api as sm
from pandas.stats.api import ols
from sklearn.linear_model import LinearRegression

import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, PercentChange, SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.factset import Fundamentals as FstFundamentals
# from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline import factors, filters, classifiers
import alphalens
import quantopian.pipeline.data.factset.estimates as fe


def signalize(df):
    return ((df.rank() - 0.5)/df.count()).replace(np.nan,0.5)

def transform(df, field, multiplier=1):
    return signalize(multiplier*df[field])

class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = close[-1] * shares[-1]

class LogMarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = np.log(close[-1] * shares[-1])

class Volatility(CustomFactor):  
    inputs = [USEquityPricing.close]  
    window_length = 25  
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0) * math.sqrt(252)

class Industry(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]

        
class IndustryGroup(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_group_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]


class Sector(CustomFactor):  
    inputs = [Fundamentals.morningstar_sector_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]

class Previous(CustomFactor):  
    # Returns value of input x trading days ago where x is the window_length  
    # Both the inputs and window_length must be specified as there are no defaults

    def compute(self, today, assets, out, inputs):  
        out[:] = inputs[0]


        
base_universe = QTradableStocksUS()

mkt_cap = MarketCap(mask = base_universe)
log_mkt_cap = LogMarketCap(mask = base_universe)
vol = Volatility(mask = base_universe)

industry = Industry(mask = base_universe)
industryGroup = IndustryGroup(mask = base_universe)
sector = Sector(mask = base_universe)


fq1_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 1)
fq1_eps_std = fq1_eps_cons.std_dev.latest

# fq1_eps_std_exsec = fq1_eps_cons.std_dev.latest.demean(mask = fq1_eps_std.notnan(), groupby=classifiers.fundamentals.Sector())

fq1_eps_mean = fq1_eps_cons.mean.latest
fq1_eps_up = fq1_eps_cons.up.latest
fq1_eps_dwn = fq1_eps_cons.down.latest
fq1_eps_num_est = fq1_eps_cons.num_est.latest


fq1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 1)
fq1_sales_std = fq1_sales_cons.std_dev.latest
fq1_sales_mean = fq1_sales_cons.mean.latest


fq0_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 0)
fq0_eps_act = fe.Actuals.slice('EPS', 'qf', 0)
fq0_eps_cons_mean = fq0_eps_cons.mean.latest
fq0_eps_act_value = fq0_eps_act.actual_value.latest

fq0_eps_surprise = (fq0_eps_act_value - fq0_eps_cons_mean)/fq0_eps_cons_mean

fq0_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 0)
fq0_sales_act = fe.Actuals.slice('SALES', 'qf', 0)
fq0_sales_cons_mean = fq0_sales_cons.mean.latest
fq0_sales_act_value = fq0_sales_act.actual_value.latest

fq0_sales_surprise = (fq0_sales_act_value - fq0_sales_cons_mean)/fq0_sales_cons_mean

#Long term
# price_tgt_cons = fe.LongTermConsensus.slice('PRICE_TGT')
# eps_gr_cons = fe.LongTermConsensus.slice('EPS_LTG')

# Get the latest mean consensus price target and EPS growth.
# price_tgt_mean = price_tgt_cons.mean.latest
# eps_gr_mean = eps_gr_cons.mean.latest
# price_tgt_std = price_tgt_cons.std_dev.latest
# eps_gr_std = eps_gr_cons.std_dev.latest

# Define an estimated price growth factor by taking the relative
# difference between the 6-12 month price target and yesterday's
# close price.
# yesterday_close = USEquityPricing.close.latest
# est_price_growth = (price_tgt_mean - yesterday_close) / yesterday_close

fe_rec = fe.ConsensusRecommendations

#It's actually SALES (check if below factor works with EPS)
guid_eps_q1 = fe.Guidance.slice('SALES', 'qf', 1)
guid_eps_q1_low = guid_eps_q1.low.latest
guid_eps_q1_high = guid_eps_q1.high.latest

guid_eps_std = (guid_eps_q1_high - guid_eps_q1_low)/(guid_eps_q1_low + guid_eps_q1_high)

#Quarterly Historical EPS/SALES growth - Actual
fq0_sales_act = fe.Actuals.slice('SALES', 'qf', 0).actual_value.latest
fqn1_sales_act = fe.Actuals.slice('SALES', 'qf', -1).actual_value.latest
fqn2_sales_act = fe.Actuals.slice('SALES', 'qf', -2).actual_value.latest
fqn3_sales_act = fe.Actuals.slice('SALES', 'qf', -3).actual_value.latest
fqn4_sales_act = fe.Actuals.slice('SALES', 'qf', -4).actual_value.latest
fqn5_sales_act = fe.Actuals.slice('SALES', 'qf', -5).actual_value.latest
fqn6_sales_act = fe.Actuals.slice('SALES', 'qf', -6).actual_value.latest
fqn7_sales_act = fe.Actuals.slice('SALES', 'qf', -7).actual_value.latest
fqn8_sales_act = fe.Actuals.slice('SALES', 'qf', -8).actual_value.latest

#Quarterly Consensus EPS/SALES
# fq1_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 1).mean.latest
# fq0_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 0).mean.latest
# fqn1_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -1).mean.latest
# fqn2_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -2).mean.latest
# fqn3_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -3).mean.latest
# fqn4_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -4).mean.latest
# fqn5_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -5).mean.latest
# fqn6_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -6).mean.latest
# fqn7_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -7).mean.latest
# fqn8_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -8).mean.latest


fq1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 1).mean.latest
fq0_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 0).mean.latest
fqn1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -1).mean.latest
fqn2_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -2).mean.latest
fqn3_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -3).mean.latest
fqn4_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -4).mean.latest
fqn5_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -5).mean.latest
fqn6_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -6).mean.latest
fqn7_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -7).mean.latest
fqn8_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -8).mean.latest


#Sales Surprise
cons_to_act_sales_surprise_q0 = fq0_sales_act - fq0_sales_cons
cons_to_act_sales_surprise_qn1 = fqn1_sales_act - fqn1_sales_cons
cons_to_act_sales_surprise_qn2 = fqn2_sales_act - fqn2_sales_cons
cons_to_act_sales_surprise_qn3 = fqn3_sales_act - fqn3_sales_cons
cons_to_act_sales_surprise_qn4 = fqn4_sales_act - fqn4_sales_cons
cons_to_act_sales_surprise_qn5 = fqn5_sales_act - fqn5_sales_cons
cons_to_act_sales_surprise_qn6 = fqn6_sales_act - fqn6_sales_cons
cons_to_act_sales_surprise_qn7 = fqn7_sales_act - fqn7_sales_cons

#EPS Guidance
guid_eps_q1_high = fe.Guidance.slice('EPS', 'qf', 1).high.latest
guid_eps_q0_high = fe.Guidance.slice('EPS', 'qf', 0).high.latest
guid_eps_qn1_high = fe.Guidance.slice('EPS', 'qf', -1).high.latest
guid_eps_qn2_high = fe.Guidance.slice('EPS', 'qf', -2).high.latest
guid_eps_qn3_high = fe.Guidance.slice('EPS', 'qf', -3).high.latest
guid_eps_qn4_high = fe.Guidance.slice('EPS', 'qf', -4).high.latest
guid_eps_qn5_high = fe.Guidance.slice('EPS', 'qf', -5).high.latest
guid_eps_qn6_high = fe.Guidance.slice('EPS', 'qf', -6).high.latest
guid_eps_qn7_high = fe.Guidance.slice('EPS', 'qf', -7).high.latest
guid_eps_qn8_high = fe.Guidance.slice('EPS', 'qf', -8).high.latest

guid_eps_q1_low = fe.Guidance.slice('EPS', 'qf', 1).low.latest
guid_eps_q0_low = fe.Guidance.slice('EPS', 'qf', 0).low.latest
guid_eps_qn1_low = fe.Guidance.slice('EPS', 'qf', -1).low.latest
guid_eps_qn2_low = fe.Guidance.slice('EPS', 'qf', -2).low.latest
guid_eps_qn3_low = fe.Guidance.slice('EPS', 'qf', -3).low.latest
guid_eps_qn4_low = fe.Guidance.slice('EPS', 'qf', -4).low.latest
guid_eps_qn5_low = fe.Guidance.slice('EPS', 'qf', -5).low.latest
guid_eps_qn6_low = fe.Guidance.slice('EPS', 'qf', -6).low.latest
guid_eps_qn7_low = fe.Guidance.slice('EPS', 'qf', -7).low.latest
guid_eps_qn8_low = fe.Guidance.slice('EPS', 'qf', -8).low.latest

#Sales Guidance
guid_sales_q1_high = fe.Guidance.slice('SALES', 'qf', 1).high.latest
guid_sales_q0_high = fe.Guidance.slice('SALES', 'qf', 0).high.latest
guid_sales_qn1_high = fe.Guidance.slice('SALES', 'qf', -1).high.latest
guid_sales_qn2_high = fe.Guidance.slice('SALES', 'qf', -2).high.latest
guid_sales_qn3_high = fe.Guidance.slice('SALES', 'qf', -3).high.latest
guid_sales_qn4_high = fe.Guidance.slice('SALES', 'qf', -4).high.latest
guid_sales_qn5_high = fe.Guidance.slice('SALES', 'qf', -5).high.latest
guid_sales_qn6_high = fe.Guidance.slice('SALES', 'qf', -6).high.latest
guid_sales_qn7_high = fe.Guidance.slice('SALES', 'qf', -7).high.latest
guid_sales_qn8_high = fe.Guidance.slice('SALES', 'qf', -8).high.latest

guid_sales_q1_low = fe.Guidance.slice('SALES', 'qf', 1).low.latest
guid_sales_q0_low = fe.Guidance.slice('SALES', 'qf', 0).low.latest
guid_sales_qn1_low = fe.Guidance.slice('SALES', 'qf', -1).low.latest
guid_sales_qn2_low = fe.Guidance.slice('SALES', 'qf', -2).low.latest
guid_sales_qn3_low = fe.Guidance.slice('SALES', 'qf', -3).low.latest
guid_sales_qn4_low = fe.Guidance.slice('SALES', 'qf', -4).low.latest
guid_sales_qn5_low = fe.Guidance.slice('SALES', 'qf', -5).low.latest
guid_sales_qn6_low = fe.Guidance.slice('SALES', 'qf', -6).low.latest
guid_sales_qn7_low = fe.Guidance.slice('SALES', 'qf', -7).low.latest
guid_sales_qn8_low = fe.Guidance.slice('SALES', 'qf', -8).low.latest

#Cash Flow Guidance
# guid_cfps_q1_high = fe.Guidance.slice('CFPS', 'qf', 1).high.latest
# guid_cfps_q0_high = fe.Guidance.slice('CFPS', 'qf', 0).high.latest
# guid_cfps_qn1_high = fe.Guidance.slice('CFPS', 'qf', -1).high.latest
# guid_cfps_qn2_high = fe.Guidance.slice('CFPS', 'qf', -2).high.latest
# guid_cfps_qn3_high = fe.Guidance.slice('CFPS', 'qf', -3).high.latest
# guid_cfps_qn4_high = fe.Guidance.slice('CFPS', 'qf', -4).high.latest
# guid_cfps_qn5_high = fe.Guidance.slice('CFPS', 'qf', -5).high.latest
# guid_cfps_qn6_high = fe.Guidance.slice('CFPS', 'qf', -6).high.latest
# guid_cfps_qn7_high = fe.Guidance.slice('CFPS', 'qf', -7).high.latest
# guid_cfps_qn8_high = fe.Guidance.slice('CFPS', 'qf', -8).high.latest

# guid_cfps_q1_low = fe.Guidance.slice('CFPS', 'qf', 1).low.latest
# guid_cfps_q0_low = fe.Guidance.slice('CFPS', 'qf', 0).low.latest
# guid_cfps_qn1_low = fe.Guidance.slice('CFPS', 'qf', -1).low.latest
# guid_cfps_qn2_low = fe.Guidance.slice('CFPS', 'qf', -2).low.latest
# guid_cfps_qn3_low = fe.Guidance.slice('CFPS', 'qf', -3).low.latest
# guid_cfps_qn4_low = fe.Guidance.slice('CFPS', 'qf', -4).low.latest
# guid_cfps_qn5_low = fe.Guidance.slice('CFPS', 'qf', -5).low.latest
# guid_cfps_qn6_low = fe.Guidance.slice('CFPS', 'qf', -6).low.latest
# guid_cfps_qn7_low = fe.Guidance.slice('CFPS', 'qf', -7).low.latest
# guid_cfps_qn8_low = fe.Guidance.slice('CFPS', 'qf', -8).low.latest


#Consistency of Sales Growth from Consensus vs Actual
#W
act_to_cons_sales_growth_q0 = (fq1_sales_cons/fq0_sales_act)  
act_to_cons_sales_growth_qn1 = (fq0_sales_cons/fqn1_sales_act)
act_to_cons_sales_growth_qn2 = (fqn1_sales_cons/fqn2_sales_act)
act_to_cons_sales_growth_qn3 = (fqn2_sales_cons/fqn3_sales_act)
act_to_cons_sales_growth_qn4 = (fqn3_sales_cons/fqn4_sales_act)
act_to_cons_sales_growth_qn5 = (fqn4_sales_cons/fqn5_sales_act)
act_to_cons_sales_growth_qn6 = (fqn5_sales_cons/fqn6_sales_act)
act_to_cons_sales_growth_qn7 = (fqn6_sales_cons/fqn7_sales_act)
act_to_cons_sales_growth_qn8 = (fqn7_sales_cons/fqn8_sales_act)


fq1_sales_guid = (guid_sales_q1_low + guid_sales_q1_high)/2
fq0_sales_guid = (guid_sales_q0_low + guid_sales_q0_high)/2
fqn1_sales_guid = (guid_sales_qn1_low + guid_sales_qn1_high)/2
fqn2_sales_guid = (guid_sales_qn2_low + guid_sales_qn2_high)/2
fqn3_sales_guid = (guid_sales_qn3_low + guid_sales_qn3_high)/2
fqn4_sales_guid = (guid_sales_qn4_low + guid_sales_qn4_high)/2
fqn5_sales_guid = (guid_sales_qn5_low + guid_sales_qn5_high)/2
fqn6_sales_guid = (guid_sales_qn6_low + guid_sales_qn6_high)/2
fqn7_sales_guid = (guid_sales_qn7_low + guid_sales_qn7_high)/2
fqn8_sales_guid = (guid_sales_qn8_low + guid_sales_qn8_high)/2

#EPS Guidance - high/low average (backstopped)
fq1_eps_guid = (guid_eps_q1_low + guid_eps_q1_high)/2
fq0_eps_guid = (guid_eps_q0_low + guid_eps_q0_high)/2
fqn1_eps_guid = (guid_eps_qn1_low + guid_eps_qn1_high)/2
fqn2_eps_guid = (guid_eps_qn2_low + guid_eps_qn2_high)/2
fqn3_eps_guid = (guid_eps_qn3_low + guid_eps_qn3_high)/2
fqn4_eps_guid = (guid_eps_qn4_low + guid_eps_qn4_high)/2
fqn5_eps_guid = (guid_eps_qn5_low + guid_eps_qn5_high)/2
fqn6_eps_guid = (guid_eps_qn6_low + guid_eps_qn6_high)/2
fqn7_eps_guid = (guid_eps_qn7_low + guid_eps_qn7_high)/2
fqn8_eps_guid = (guid_eps_qn8_low + guid_eps_qn8_high)/2



pipe_my_alpha_factor = Pipeline(
    columns={
        'vol': vol,
        'log_mkt_cap': log_mkt_cap,
        'mkt_cap':mkt_cap,

'fq0_sales_act':fq0_sales_act,
'fqn1_sales_act':fqn1_sales_act,
'fqn2_sales_act':fqn2_sales_act,
'fqn3_sales_act':fqn3_sales_act,
'fqn4_sales_act':fqn4_sales_act,
'fqn5_sales_act':fqn5_sales_act,
'fqn6_sales_act':fqn6_sales_act,
'fqn7_sales_act':fqn7_sales_act,
'fqn8_sales_act':fqn8_sales_act,

#EPS Guidance        
# 'guid_eps_q1_high':guid_eps_q1_high,
# 'guid_eps_q0_high':guid_eps_q0_high,
# 'guid_eps_qn1_high':guid_eps_qn1_high,
# 'guid_eps_qn2_high':guid_eps_qn2_high,
# 'guid_eps_qn3_high':guid_eps_qn3_high,
# 'guid_eps_qn4_high':guid_eps_qn4_high,
# 'guid_eps_qn5_high':guid_eps_qn5_high,
# 'guid_eps_qn6_high':guid_eps_qn6_high,
# 'guid_eps_qn7_high':guid_eps_qn7_high,
# 'guid_eps_qn8_high':guid_eps_qn8_high,

# 'guid_eps_q1_low':guid_eps_q1_low,
# 'guid_eps_q0_low':guid_eps_q0_low,
# 'guid_eps_qn1_low':guid_eps_qn1_low,
# 'guid_eps_qn2_low':guid_eps_qn2_low,
# 'guid_eps_qn3_low':guid_eps_qn3_low,
# 'guid_eps_qn4_low':guid_eps_qn4_low,
# 'guid_eps_qn5_low':guid_eps_qn5_low,
# 'guid_eps_qn6_low':guid_eps_qn6_low,
# 'guid_eps_qn7_low':guid_eps_qn7_low,
# 'guid_eps_qn8_low':guid_eps_qn8_low,
        
#Sales Guidance        
# 'guid_sales_q1_high':guid_sales_q1_high,
# 'guid_sales_q0_high':guid_sales_q0_high,
# 'guid_sales_qn1_high':guid_sales_qn1_high,
# 'guid_sales_qn2_high':guid_sales_qn2_high,
# 'guid_sales_qn3_high':guid_sales_qn3_high,
# 'guid_sales_qn4_high':guid_sales_qn4_high,
# 'guid_sales_qn5_high':guid_sales_qn5_high,
# 'guid_sales_qn6_high':guid_sales_qn6_high,
# 'guid_sales_qn7_high':guid_sales_qn7_high,
# 'guid_sales_qn8_high':guid_sales_qn8_high,

# 'guid_sales_q1_low':guid_sales_q1_low,
# 'guid_sales_q0_low':guid_sales_q0_low,
# 'guid_sales_qn1_low':guid_sales_qn1_low,
# 'guid_sales_qn2_low':guid_sales_qn2_low,
# 'guid_sales_qn3_low':guid_sales_qn3_low,
# 'guid_sales_qn4_low':guid_sales_qn4_low,
# 'guid_sales_qn5_low':guid_sales_qn5_low,
# 'guid_sales_qn6_low':guid_sales_qn6_low,
# 'guid_sales_qn7_low':guid_sales_qn7_low,
# 'guid_sales_qn8_low':guid_sales_qn8_low,
        
        
'fq1_sales_guid':fq1_sales_guid,
'fq0_sales_guid':fq0_sales_guid,
'fqn1_sales_guid':fqn1_sales_guid,
'fqn2_sales_guid':fqn2_sales_guid,
'fqn3_sales_guid':fqn3_sales_guid,
'fqn4_sales_guid':fqn4_sales_guid,
'fqn5_sales_guid':fqn5_sales_guid,
'fqn6_sales_guid':fqn6_sales_guid,
'fqn7_sales_guid':fqn7_sales_guid,
'fqn8_sales_guid':fqn8_sales_guid,

        
'fq1_eps_guid':fq1_eps_guid,
'fq0_eps_guid':fq0_eps_guid,
'fqn1_eps_guid':fqn1_eps_guid,
'fqn2_eps_guid':fqn2_eps_guid,
'fqn3_eps_guid':fqn3_eps_guid,
'fqn4_eps_guid':fqn4_eps_guid,
'fqn5_eps_guid':fqn5_eps_guid,
'fqn6_eps_guid':fqn6_eps_guid,
'fqn7_eps_guid':fqn7_eps_guid,
'fqn8_eps_guid':fqn8_eps_guid,
        
      
'fq1_sales_cons':fq1_sales_cons,
'fq0_sales_cons':fq0_sales_cons,
'fqn1_sales_cons':fqn1_sales_cons,
'fqn2_sales_cons':fqn2_sales_cons,
'fqn3_sales_cons':fqn3_sales_cons,
'fqn4_sales_cons':fqn4_sales_cons,
'fqn5_sales_cons':fqn5_sales_cons,
'fqn6_sales_cons':fqn6_sales_cons,
'fqn7_sales_cons':fqn7_sales_cons,
'fqn8_sales_cons':fqn8_sales_cons,

#Consensus to Actual sales growth - Works
'act_to_cons_sales_growth_q0': act_to_cons_sales_growth_q0,
'act_to_cons_sales_growth_qn1': act_to_cons_sales_growth_qn1,
'act_to_cons_sales_growth_qn2': act_to_cons_sales_growth_qn2,
'act_to_cons_sales_growth_qn3': act_to_cons_sales_growth_qn3,
'act_to_cons_sales_growth_qn4': act_to_cons_sales_growth_qn4,
'act_to_cons_sales_growth_qn5': act_to_cons_sales_growth_qn5,
'act_to_cons_sales_growth_qn6': act_to_cons_sales_growth_qn6,
'act_to_cons_sales_growth_qn7': act_to_cons_sales_growth_qn7,
'act_to_cons_sales_growth_qn8': act_to_cons_sales_growth_qn8,

                
        'fq1_eps_std': fq1_eps_std,
        'fq1_eps_up':fq1_eps_up,
        'fq1_eps_dwn':fq1_eps_dwn,
        'fq1_eps_num_est':fq1_eps_num_est,
        'fq1_sales_mean': fq1_sales_mean,
        'fq1_sales_std': fq1_sales_std,

        #CONSISTENCY WORKS
        'cons_to_act_sales_surprise_q0': cons_to_act_sales_surprise_q0,
        'cons_to_act_sales_surprise_qn1': cons_to_act_sales_surprise_qn1,
        'cons_to_act_sales_surprise_qn2': cons_to_act_sales_surprise_qn2,
        'cons_to_act_sales_surprise_qn3': cons_to_act_sales_surprise_qn3,
        'cons_to_act_sales_surprise_qn4': cons_to_act_sales_surprise_qn4,
        'cons_to_act_sales_surprise_qn5': cons_to_act_sales_surprise_qn5,
        'cons_to_act_sales_surprise_qn6': cons_to_act_sales_surprise_qn6,
        'cons_to_act_sales_surprise_qn7': cons_to_act_sales_surprise_qn7,
            
# next quarter guidance = guidance q1 - actuals q0
# next quarter guidance = guidance q1 - actuals q0 * growth from qn1 to q0
        

    },
    screen=base_universe
)

output = run_pipeline(pipe_my_alpha_factor, '2015-06-01', '2018-09-01')

print output.head(5)


In [ ]:
#Sales Guidance - high/low average (backstopped)
# output['fq1_sales_guid'] = ((output['guid_sales_q1_low'] + output['guid_sales_q1_high'])/2).fillna(output['guid_sales_q1_high']).fillna(output['guid_sales_q1_low'])  
# output.drop(['guid_sales_q1_low','guid_sales_q1_high'], axis=1)

# output['fq0_sales_guid'] = ((output['guid_sales_q0_low'] + output['guid_sales_q0_high'])/2).fillna(output['guid_sales_q0_high']).fillna(output['guid_sales_q0_low'])  
# output.drop(['guid_sales_q0_low','guid_sales_q0_high'], axis=1)

# output['fqn1_sales_guid'] = ((output['guid_sales_qn1_low'] + output['guid_sales_qn1_high'])/2).fillna(output['guid_sales_qn1_high']).fillna(output['guid_sales_qn1_low'])  
# output.drop(['guid_sales_qn1_low','guid_sales_qn1_high'], axis=1)

# output['fqn2_sales_guid'] = ((output['guid_sales_qn2_low'] + output['guid_sales_qn2_high'])/2).fillna(output['guid_sales_qn2_high']).fillna(output['guid_sales_qn2_low'])  
# output.drop(['guid_sales_qn2_low','guid_sales_qn2_high'], axis=1)

# output['fqn3_sales_guid'] = ((output['guid_sales_qn3_low'] + output['guid_sales_qn3_high'])/2).fillna(output['guid_sales_qn3_high']).fillna(output['guid_sales_qn3_low'])  
# output.drop(['guid_sales_qn3_low','guid_sales_qn3_high'], axis=1)

# output['fqn4_sales_guid'] = ((output['guid_sales_qn4_low'] + output['guid_sales_qn4_high'])/2).fillna(output['guid_sales_qn4_high']).fillna(output['guid_sales_qn4_low'])  
# output.drop(['guid_sales_qn4_low','guid_sales_qn4_high'], axis=1)

# output['fqn5_sales_guid'] = ((output['guid_sales_qn5_low'] + output['guid_sales_qn5_high'])/2).fillna(output['guid_sales_qn5_high']).fillna(output['guid_sales_qn5_low'])  
# output.drop(['guid_sales_qn5_low','guid_sales_qn5_high'], axis=1)

# output['fqn6_sales_guid'] = ((output['guid_sales_qn6_low'] + output['guid_sales_qn6_high'])/2).fillna(output['guid_sales_qn6_high']).fillna(output['guid_sales_qn6_low'])  
# output.drop(['guid_sales_qn6_low','guid_sales_qn6_high'], axis=1)

# output['fqn7_sales_guid'] = ((output['guid_sales_qn7_low'] + output['guid_sales_qn7_high'])/2).fillna(output['guid_sales_qn7_high']).fillna(output['guid_sales_qn7_low'])  
# output.drop(['guid_sales_qn7_low','guid_sales_qn7_high'], axis=1)

# output['fqn8_sales_guid'] = ((output['guid_sales_qn8_low'] + output['guid_sales_qn8_high'])/2).fillna(output['guid_sales_qn8_high']).fillna(output['guid_sales_qn8_low'])  
# output.drop(['guid_sales_qn8_low','guid_sales_qn8_high'], axis=1)

# #EPS Guidance - high/low average (backstopped)
# output['fq1_eps_guid'] = ((output['guid_eps_q1_low'] + output['guid_eps_q1_high'])/2).fillna(output['guid_eps_q1_high']).fillna(output['guid_eps_q1_low'])  
# output.drop(['guid_eps_q1_low','guid_eps_q1_high'], axis=1)

# output['fq0_eps_guid'] = ((output['guid_eps_q0_low'] + output['guid_eps_q0_high'])/2).fillna(output['guid_eps_q0_high']).fillna(output['guid_eps_q0_low'])  
# output.drop(['guid_eps_q0_low','guid_eps_q1_high'], axis=1)

# output['fqn1_eps_guid'] = ((output['guid_eps_qn1_low'] + output['guid_eps_qn1_high'])/2).fillna(output['guid_eps_qn1_high']).fillna(output['guid_eps_qn1_low'])  
# output.drop(['guid_eps_qn1_low','guid_eps_qn1_high'], axis=1)

# output['fqn2_eps_guid'] = ((output['guid_eps_qn2_low'] + output['guid_eps_qn2_high'])/2).fillna(output['guid_eps_qn2_high']).fillna(output['guid_eps_qn2_low'])  
# output.drop(['guid_eps_qn2_low','guid_eps_qn2_high'], axis=1)

# output['fqn3_eps_guid'] = ((output['guid_eps_qn3_low'] + output['guid_eps_qn3_high'])/2).fillna(output['guid_eps_qn3_high']).fillna(output['guid_eps_qn3_low'])  
# output.drop(['guid_eps_qn3_low','guid_eps_qn3_high'], axis=1)

# output['fqn4_eps_guid'] = ((output['guid_eps_qn4_low'] + output['guid_eps_qn4_high'])/2).fillna(output['guid_eps_qn4_high']).fillna(output['guid_eps_qn4_low'])  
# output.drop(['guid_eps_qn4_low','guid_eps_qn4_high'], axis=1)

# output['fqn5_eps_guid'] = ((output['guid_eps_qn5_low'] + output['guid_eps_qn5_high'])/2).fillna(output['guid_eps_qn5_high']).fillna(output['guid_eps_qn5_low'])  
# output.drop(['guid_eps_qn5_low','guid_eps_qn5_high'], axis=1)

# output['fqn6_eps_guid'] = ((output['guid_eps_qn6_low'] + output['guid_eps_qn6_high'])/2).fillna(output['guid_eps_qn6_high']).fillna(output['guid_eps_qn6_low'])  
# output.drop(['guid_eps_qn6_low','guid_eps_qn6_high'], axis=1)

# output['fqn7_eps_guid'] = ((output['guid_eps_qn7_low'] + output['guid_eps_qn7_high'])/2).fillna(output['guid_eps_qn7_high']).fillna(output['guid_eps_qn7_low'])  
# output.drop(['guid_eps_qn7_low','guid_eps_qn7_high'], axis=1)

# output['fqn8_eps_guid'] = ((output['guid_eps_qn8_low'] + output['guid_eps_qn8_high'])/2).fillna(output['guid_eps_qn8_high']).fillna(output['guid_eps_qn8_low'])  
# output.drop(['guid_eps_qn8_low','guid_eps_qn8_high'], axis=1)


#################
# output['fq1_sales_guid'] = output['guid_sales_q1_high'].fillna(output['guid_sales_q1_low'])  
# output.drop(['guid_sales_q1_low','guid_sales_q1_high'], axis=1)

# output['fq0_sales_guid'] = output['guid_sales_q0_high'].fillna(output['guid_sales_q0_low'])  
# output.drop(['guid_sales_q0_low','guid_sales_q0_high'], axis=1)

# output['fqn1_sales_guid'] = output['guid_sales_qn1_high'].fillna(output['guid_sales_qn1_low'])  
# output.drop(['guid_sales_qn1_low','guid_sales_qn1_high'], axis=1)

# output['fqn2_sales_guid'] = output['guid_sales_qn2_high'].fillna(output['guid_sales_qn2_low'])  
# output.drop(['guid_sales_qn2_low','guid_sales_qn2_high'], axis=1)

# output['fqn3_sales_guid'] = output['guid_sales_qn3_high'].fillna(output['guid_sales_qn3_low'])  
# output.drop(['guid_sales_qn3_low','guid_sales_qn3_high'], axis=1)

# output['fqn4_sales_guid'] = output['guid_sales_qn4_high'].fillna(output['guid_sales_qn4_low'])  
# output.drop(['guid_sales_qn4_low','guid_sales_qn4_high'], axis=1)

# output['fqn5_sales_guid'] = output['guid_sales_qn5_high'].fillna(output['guid_sales_qn5_low'])  
# output.drop(['guid_sales_qn5_low','guid_sales_qn5_high'], axis=1)

# output['fqn6_sales_guid'] = output['guid_sales_qn6_high'].fillna(output['guid_sales_qn6_low'])  
# output.drop(['guid_sales_qn6_low','guid_sales_qn6_high'], axis=1)

# output['fqn7_sales_guid'] = output['guid_sales_qn7_high'].fillna(output['guid_sales_qn7_low'])  
# output.drop(['guid_sales_qn7_low','guid_sales_qn7_high'], axis=1)

# output['fqn8_sales_guid'] = output['guid_sales_qn8_high'].fillna(output['guid_sales_qn8_low'])  
# output.drop(['guid_sales_qn8_low','guid_sales_qn8_high'], axis=1)

# #EPS Guidance - high/low average (backstopped)
# output['fq1_eps_guid'] = output['guid_eps_q1_high'].fillna(output['guid_eps_q1_low'])  
# output.drop(['guid_eps_q1_low','guid_eps_q1_high'], axis=1)

# output['fq0_eps_guid'] = output['guid_eps_q0_high'].fillna(output['guid_eps_q0_low'])  
# output.drop(['guid_eps_q0_low','guid_eps_q1_high'], axis=1)

# output['fqn1_eps_guid'] = output['guid_eps_qn1_high'].fillna(output['guid_eps_qn1_low'])  
# output.drop(['guid_eps_qn1_low','guid_eps_qn1_high'], axis=1)

# output['fqn2_eps_guid'] = output['guid_eps_qn2_high'].fillna(output['guid_eps_qn2_low'])  
# output.drop(['guid_eps_qn2_low','guid_eps_qn2_high'], axis=1)

# output['fqn3_eps_guid'] = output['guid_eps_qn3_high'].fillna(output['guid_eps_qn3_low'])  
# output.drop(['guid_eps_qn3_low','guid_eps_qn3_high'], axis=1)

# output['fqn4_eps_guid'] = output['guid_eps_qn4_high'].fillna(output['guid_eps_qn4_low'])  
# output.drop(['guid_eps_qn4_low','guid_eps_qn4_high'], axis=1)

# output['fqn5_eps_guid'] = output['guid_eps_qn5_high'].fillna(output['guid_eps_qn5_low'])  
# output.drop(['guid_eps_qn5_low','guid_eps_qn5_high'], axis=1)

# output['fqn6_eps_guid'] = output['guid_eps_qn6_high'].fillna(output['guid_eps_qn6_low'])  
# output.drop(['guid_eps_qn6_low','guid_eps_qn6_high'], axis=1)

# output['fqn7_eps_guid'] = output['guid_eps_qn7_high'].fillna(output['guid_eps_qn7_low'])  
# output.drop(['guid_eps_qn7_low','guid_eps_qn7_high'], axis=1)

# output['fqn8_eps_guid'] = output['guid_eps_qn8_high'].fillna(output['guid_eps_qn8_low'])  
# output.drop(['guid_eps_qn8_low','guid_eps_qn8_high'], axis=1)



# #Delete the guidance low/high columns
# output.drop(['guid_eps_q1_low',
#                      'guid_eps_q0_low',
#                      'guid_eps_qn1_low',
#                      'guid_eps_qn2_low',
#                      'guid_eps_qn3_low',
#                      'guid_eps_qn4_low',
#                      'guid_eps_qn5_low',
#                      'guid_eps_qn6_low',
#                      'guid_eps_qn7_low',
#                      'guid_eps_qn8_low',
#                      'guid_eps_q1_high',
#                      'guid_eps_q0_high',
#                      'guid_eps_qn1_high',
#                      'guid_eps_qn2_high',
#                      'guid_eps_qn3_high',
#                      'guid_eps_qn4_high',
#                      'guid_eps_qn5_high',
#                      'guid_eps_qn6_high',
#                      'guid_eps_qn7_high',
#                      'guid_eps_qn8_high',
#                      'guid_sales_q1_low',
#                      'guid_sales_q0_low',
#                      'guid_sales_qn1_low',
#                      'guid_sales_qn2_low',
#                      'guid_sales_qn3_low',
#                      'guid_sales_qn4_low',
#                      'guid_sales_qn5_low',
#                      'guid_sales_qn6_low',
#                      'guid_sales_qn7_low',
#                      'guid_sales_qn8_low',
#                      'guid_sales_q1_high',
#                      'guid_sales_q0_high',
#                      'guid_sales_qn1_high',
#                      'guid_sales_qn2_high',
#                      'guid_sales_qn3_high',
#                      'guid_sales_qn4_high',
#                      'guid_sales_qn5_high',
#                      'guid_sales_qn6_high',
#                      'guid_sales_qn7_high',
#                      'guid_sales_qn8_high'
#                     ], axis=1)


#Sales surprise over guidance - Works
output['guid_to_act_sales_surprise_q0'] = output['fq0_sales_act'] - output['fq0_sales_guid'] 
output['guid_to_act_sales_surprise_qn1'] = output['fqn1_sales_act'] - output['fqn1_sales_guid']
output['guid_to_act_sales_surprise_qn2'] = output['fqn2_sales_act'] - output['fqn2_sales_guid']
output['guid_to_act_sales_surprise_qn3'] = output['fqn3_sales_act'] - output['fqn3_sales_guid']
output['guid_to_act_sales_surprise_qn4'] = output['fqn4_sales_act'] - output['fqn4_sales_guid']
output['guid_to_act_sales_surprise_qn5'] = output['fqn5_sales_act'] - output['fqn5_sales_guid']
output['guid_to_act_sales_surprise_qn6'] = output['fqn6_sales_act'] - output['fqn6_sales_guid']
output['guid_to_act_sales_surprise_qn7'] = output['fqn7_sales_act'] - output['fqn7_sales_guid']
output['guid_to_act_sales_surprise_qn8'] = output['fqn8_sales_act'] - output['fqn8_sales_guid']

#Consensus to Guidance Sales Surprise - Works
output['cons_to_guid_sales_surprise_q1'] = output['fq1_sales_guid'] - output['fq1_sales_cons'] 
output['cons_to_guid_sales_surprise_q0'] = output['fq0_sales_guid'] - output['fq0_sales_cons'] 
output['cons_to_guid_sales_surprise_qn1'] = output['fqn1_sales_guid'] - output['fqn1_sales_cons']
output['cons_to_guid_sales_surprise_qn2'] = output['fqn2_sales_guid'] - output['fqn2_sales_cons']
output['cons_to_guid_sales_surprise_qn3'] = output['fqn3_sales_guid'] - output['fqn3_sales_cons']
output['cons_to_guid_sales_surprise_qn4'] = output['fqn4_sales_guid'] - output['fqn4_sales_cons']
output['cons_to_guid_sales_surprise_qn5'] = output['fqn5_sales_guid'] - output['fqn5_sales_cons']
output['cons_to_guid_sales_surprise_qn6'] = output['fqn6_sales_guid'] - output['fqn6_sales_cons']
output['cons_to_guid_sales_surprise_qn7'] = output['fqn7_sales_guid'] - output['fqn7_sales_cons']
output['cons_to_guid_sales_surprise_qn8'] = output['fqn8_sales_guid'] - output['fqn8_sales_cons']


#GROWTH
#Consensus to Guidance Sales Growth - Works
output['cons_to_guid_sales_growth_q0'] = output['fq1_sales_guid']/output['fq0_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn1'] = output['fq0_sales_guid']/output['fqn1_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn2'] = output['fqn1_sales_guid']/output['fqn2_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn3'] = output['fqn2_sales_guid']/output['fqn3_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn4'] = output['fqn3_sales_guid']/output['fqn4_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn5'] = output['fqn4_sales_guid']/output['fqn5_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn6'] = output['fqn5_sales_guid']/output['fqn6_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn7'] = output['fqn6_sales_guid']/output['fqn7_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn8'] = output['fqn7_sales_guid']/output['fqn8_sales_cons'] - 1


output['guid_to_guid_sales_growth_q0'] = output['fq1_sales_guid']/output['fq0_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn1'] = output['fq0_sales_guid']/output['fqn1_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn2'] = output['fqn1_sales_guid']/output['fqn2_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn3'] = output['fqn2_sales_guid']/output['fqn3_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn4'] = output['fqn3_sales_guid']/output['fqn4_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn5'] = output['fqn4_sales_guid']/output['fqn5_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn6'] = output['fqn5_sales_guid']/output['fqn6_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn7'] = output['fqn6_sales_guid']/output['fqn7_sales_guid'] - 1
output['guid_to_guid_sales_growth_qn8'] = output['fqn7_sales_guid']/output['fqn8_sales_guid'] - 1


# Consensus to Guidance Cash Flow Growth
# output['cons_to_guid_cfps_growth_q0'] = output['fq1_cfps_guid']/output['fq0_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn1'] = output['fq0_cfps_guid']/output['fqn1_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn2'] = output['fqn1_cfps_guid']/output['fqn2_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn3'] = output['fqn2_cfps_guid']/output['fqn3_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn4'] = output['fqn3_cfps_guid']/output['fqn4_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn5'] = output['fqn4_cfps_guid']/output['fqn5_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn6'] = output['fqn5_cfps_guid']/output['fqn6_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn7'] = output['fqn6_cfps_guid']/output['fqn7_cfps_cons'] - 1
# output['cons_to_guid_cfps_growth_qn8'] = output['fqn7_cfps_guid']/output['fqn8_cfps_cons'] - 1

#Consensus to Guidance EPS Growth
# output['cons_to_guid_eps_growth_q0'] = output['fq1_eps_guid']/output['fq0_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn1'] = output['fq0_eps_guid']/output['fqn1_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn2'] = output['fqn1_eps_guid']/output['fqn2_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn3'] = output['fqn2_eps_guid']/output['fqn3_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn4'] = output['fqn3_eps_guid']/output['fqn4_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn5'] = output['fqn4_eps_guid']/output['fqn5_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn6'] = output['fqn5_eps_guid']/output['fqn6_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn7'] = output['fqn6_eps_guid']/output['fqn7_eps_cons'] - 1
# output['cons_to_guid_eps_growth_qn8'] = output['fqn7_eps_guid']/output['fqn8_eps_cons'] - 1

In [ ]:
# W
alpha1 = signalize(1/output['fq1_eps_std']).dropna()

#W
alpha2 = signalize((output['fq1_eps_up'] - output['fq1_eps_dwn'])/output['fq1_eps_num_est']).dropna()
#W
alpha3 = signalize(output['fq1_sales_mean']/output['fq1_sales_std']).dropna()

#W
alpha9 = signalize(output['cons_to_act_sales_surprise_q0']).dropna()

alpha_s1 = alpha1 + alpha2 + alpha3 + alpha9 

cons_to_act_sales_surprise_std = output[[
    'cons_to_act_sales_surprise_q0',
    'cons_to_act_sales_surprise_qn1', 
    'cons_to_act_sales_surprise_qn2',
    'cons_to_act_sales_surprise_qn3',
    'cons_to_act_sales_surprise_qn4',
    'cons_to_act_sales_surprise_qn5',
    'cons_to_act_sales_surprise_qn6',
    'cons_to_act_sales_surprise_qn7',
]].std(axis=1)

cons_to_act_sales_surprise_mean = output[[
    'cons_to_act_sales_surprise_q0',
    'cons_to_act_sales_surprise_qn1', 
    'cons_to_act_sales_surprise_qn2',
    'cons_to_act_sales_surprise_qn3',
    'cons_to_act_sales_surprise_qn4',
    'cons_to_act_sales_surprise_qn5',
    'cons_to_act_sales_surprise_qn6',
    'cons_to_act_sales_surprise_qn7',
]].mean(axis=1)

alpha38 = signalize(cons_to_act_sales_surprise_mean/cons_to_act_sales_surprise_std)

guid_to_act_sales_surprise_std = output[[
    'guid_to_act_sales_surprise_q0',
    'guid_to_act_sales_surprise_qn1', 
    'guid_to_act_sales_surprise_qn2',
    'guid_to_act_sales_surprise_qn3',
    'guid_to_act_sales_surprise_qn4',
    'guid_to_act_sales_surprise_qn5',
    'guid_to_act_sales_surprise_qn6',
    'guid_to_act_sales_surprise_qn7',
    'guid_to_act_sales_surprise_qn8'
]].std(axis=1)

guid_to_act_sales_surprise_mean = output[[
    'guid_to_act_sales_surprise_q0',
    'guid_to_act_sales_surprise_qn1', 
    'guid_to_act_sales_surprise_qn2',
    'guid_to_act_sales_surprise_qn3',
    'guid_to_act_sales_surprise_qn4',
    'guid_to_act_sales_surprise_qn5',
    'guid_to_act_sales_surprise_qn6',
    'guid_to_act_sales_surprise_qn7',
    'guid_to_act_sales_surprise_qn8'
]].mean(axis=1)

alpha39 = signalize(guid_to_act_sales_surprise_std/guid_to_act_sales_surprise_std)


cs_sales_mean_q = output[['fq0_sales_guid', 
'fqn1_sales_guid',
'fqn2_sales_guid',
'fqn3_sales_guid',
'fqn4_sales_guid',
'fqn5_sales_guid',
'fqn7_sales_guid',
'fqn8_sales_guid']].mean(axis=1)

cs_sales_std_q = output[['fq0_sales_guid', 
'fqn1_sales_guid',
'fqn2_sales_guid',
'fqn3_sales_guid',
'fqn4_sales_guid',
'fqn5_sales_guid',
'fqn7_sales_guid',
'fqn8_sales_guid']].std(axis=1)

#W
alpha34 = signalize(cs_sales_mean_q/cs_sales_std_q)

alpha361 = signalize((output['fq0_sales_guid'] - output['fqn3_sales_guid'])/output['fqn3_sales_guid'])
alpha362 = signalize((output['fq0_sales_guid'] - output['fqn2_sales_guid'])/output['fqn2_sales_guid'])
alpha36 = (alpha361+alpha362)/2

alpha_s1 = alpha_s1.sub(alpha_s1.mean(level=0), level=0)


cs_act_to_cons_sales_growth_mean = output[['act_to_cons_sales_growth_q0', 
                                       'act_to_cons_sales_growth_qn1',
                                        'act_to_cons_sales_growth_qn2',
                                        'act_to_cons_sales_growth_qn3',
                                        'act_to_cons_sales_growth_qn4',
                                        'act_to_cons_sales_growth_qn5',
                                        'act_to_cons_sales_growth_qn6',
                                        'act_to_cons_sales_growth_qn7',
                                       'act_to_cons_sales_growth_qn8',
                                       ]].mean(axis=1) 

cs_act_to_cons_sales_growth_std = output[['act_to_cons_sales_growth_q0', 
                                        'act_to_cons_sales_growth_qn1',
                                       'act_to_cons_sales_growth_qn2',
                                        'act_to_cons_sales_growth_qn3',
                                        'act_to_cons_sales_growth_qn4',
                                        'act_to_cons_sales_growth_qn5',
                                        'act_to_cons_sales_growth_qn6',
                                        'act_to_cons_sales_growth_qn7',
                                      'act_to_cons_sales_growth_qn8',
                                       ]].std(axis=1) 


alpha411 = signalize(cs_act_to_cons_sales_growth_mean)
alpha412 = signalize(-cs_act_to_cons_sales_growth_std)
alpha413 = signalize(cs_act_to_cons_sales_growth_mean/cs_act_to_cons_sales_growth_std)


#Sales Suprise interacted with efficiency
efficiency = signalize(signalize(-output['log_mkt_cap']) + signalize(output['vol']) + signalize(-output['fq1_eps_num_est']))

cs_cons_to_guid_sales_surprise_mean = output[['cons_to_guid_sales_surprise_q1',
                                    'cons_to_guid_sales_surprise_q0',
                                    'cons_to_guid_sales_surprise_qn1',
                                    'cons_to_guid_sales_surprise_qn2',
                                    'cons_to_guid_sales_surprise_qn3',
                                    'cons_to_guid_sales_surprise_qn4',
                                    'cons_to_guid_sales_surprise_qn5',
                                    'cons_to_guid_sales_surprise_qn6',
                                    'cons_to_guid_sales_surprise_qn7',
                                         ]].mean(axis=1)

cs_cons_to_guid_sales_surprise_std = output[['cons_to_guid_sales_surprise_q1',
                                    'cons_to_guid_sales_surprise_q0',
                                    'cons_to_guid_sales_surprise_qn1',
                                    'cons_to_guid_sales_surprise_qn2',
                                    'cons_to_guid_sales_surprise_qn3',
                                    'cons_to_guid_sales_surprise_qn4',
                                    'cons_to_guid_sales_surprise_qn5',
                                    'cons_to_guid_sales_surprise_qn6',
                                    'cons_to_guid_sales_surprise_qn7',
                                         ]].std(axis=1)


alpha431 = signalize(cs_cons_to_guid_sales_surprise_mean) 
alpha432 = signalize(-cs_cons_to_guid_sales_surprise_std) 
alpha433 = signalize(cs_cons_to_guid_sales_surprise_mean/cs_cons_to_guid_sales_surprise_std) 


alpha49 = signalize(-output[[
            'fq1_eps_guid',
            'fq0_eps_guid',
            'fqn1_eps_guid',
            'fqn2_eps_guid',
            'fqn3_eps_guid',
            'fqn4_eps_guid',
            'fqn5_eps_guid',
            'fqn6_eps_guid',
            'fqn7_eps_guid',
            'fqn8_eps_guid']].std(axis=1))

alpha50 = signalize((output['fq1_sales_guid'] - output['fq0_sales_guid'])/output['fq0_sales_guid'])

In [ ]:
# print efficiency.head()
print alpha432.head()

In [ ]:
# print alpha41.head()
output['alpha1'] = alpha1
output['alpha2'] = alpha2
output['alpha3'] = alpha3
output['alpha9'] = alpha9
output['alpha38'] = alpha38
output['alpha39'] = alpha39
output['alpha34'] = alpha34
output['alpha36'] = alpha36
output['alpha412'] = alpha412
output['alpha432'] = alpha432

output['alpha49'] = alpha49
output['alpha50'] = alpha50

# output['weights'] = 1 + 0.3*output['vol_rank'] + 0.7*output['mk_rank']
# print output['alpha411']


In [ ]:
def wmean(name):
    return (output[name]*output['weights']).sum(level=0)/output['weights'].sum(level=0)

alphas = pd.DataFrame()

for name in ['alpha1','alpha2','alpha3','alpha9','alpha38','alpha39',
             'alpha34','alpha36',
             'alpha412',
             'alpha432',
             'alpha49', 'alpha50'
            ]:
    alphas[name] = output[name].sub(output[name].mean(level=0), level=0)
#     alphas[name] = output[name].sub(wmean(name), level=0)

# alphas['weights'] = output['weights']    

In [ ]:
assets = output.index.levels[1].unique()
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
pricing = get_pricing(assets, start_date='2015-06-01', end_date='2018-09-01', fields='close_price')
print "DONE Pricing"


In [ ]:
returns = pricing.pct_change(periods=5).shift(-5)
returns = returns.sub(returns.mean(axis=1), axis=0).stack()
# r_m = returns.mean(level=0)
# print returns

data = pd.concat([returns.rename('returns'), alphas], axis=1).dropna()

data.reset_index(inplace=True)
data = data.rename(columns={'level_0':'dates', 'level_1': 'sid'})
# print data.head()

In [ ]:
data['d1'] = data['alpha1'] 
data['d2'] = data['alpha1'] + data['alpha2'] 
data['d3'] = data['alpha1'] + data['alpha2'] + data['alpha3'] 
data['d4'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38']
data['d5'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] 
# data['d6'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha40']
data['d7'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']
data['d8'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha36']
# data['d9'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha40']
data['d10'] = data['alpha1'] + data['alpha2'] + 2*data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']
data['d11'] = data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha412']
data['d12'] = data['d8'] + data['alpha412'] 
data['d13'] = data['d12'] + data['alpha432']  
data['d14'] = data['d13'] + data['alpha49'] 
data['d15'] = data['d14'] + data['alpha50']

data['d16'] = data['d8'] + data['alpha412'] + data['alpha432']  +  data['alpha49'] + data['alpha50']
data['d17'] = data['d4'] + data['alpha412'] + data['alpha432']  +  data['alpha49'] + data['alpha50']

est = sm.OLS(
    100*data[['returns']], 
#         data[['alpha412', 'alpha432', 'alpha49', 'alpha50']])
        data[['d17']])
est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()


In [ ]:
print data['alpha412'].corr(data['alpha49'])

In [ ]:
# print data
# print data['returns'].shape
# data = data.reset_index()
# print data[['alpha']]

# reg = LinearRegression().fit(
# data[['returns']],
# data[['alpha1','alpha2', 'alpha3', 'alpha9', 'alpha38', 'alpha32']])

# data['d1'] = data['alpha1'] 
# data['d2'] = data['alpha1'] + data['alpha2'] 
# data['d3'] = data['alpha1'] + data['alpha2'] + data['alpha3'] 
# data['d4'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38']
# data['d5'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39']
# data['d6'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha40']
# data['d7'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']
# data['d8'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha36']
# data['d9'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha40']
# data['d10'] = data['alpha1'] + data['alpha2'] + 2*data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']

# est = sm.WLS(
#     100*data[['returns']], 
#             data[['d7']],
#     weights=data[['weights']])
# est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

# print est2.summary()



In [ ]:
print (data['d4']).corr(data['alpha432'] + data['alpha49'] + data['alpha50'])

In [ ]:
# coef =  reg.coef_
# print est2.params
# net_alpha =  data[['alpha1', 'alpha2', 'alpha3', 'alpha9', 'alpha38']].dot(est2.params.T)/np.sum(np.abs(est2.params.T))
# net_alpha = alpha_s1 + alpha38 + alpha39 + alpha34 + alpha36 + alpha40 
# net_alpha = alpha_s1 - alpha9 + alpha38 + alpha39 + alpha40 
# net_alpha = alpha_s1 + alpha38 + alpha39 + alpha34 + alpha36 
# net_alpha = alpha_s1 + alpha38 + alpha39 + alpha34 + alpha36 + alpha412 
# net_alpha = alpha_s1 + alpha38 + alpha39 + alpha34 + alpha36 + alpha412 + alpha432 + alpha49 + alpha50
net_alpha = alpha1 + alpha2 + alpha3 + alpha38 + alpha412 + alpha432 + alpha49 + alpha50
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)

In [ ]:
#List of factors that dont't work
#Sales Yield - Guidance
# fq0_sales_yield_guid = fq0_sales_guid/(fq0_sales_guid - fq0_eps_guid * Fundamentals.shares_outstanding)
# fqn1_sales_yield_guid = fqn1_sales_guid/(fqn1_sales_guid - fqn1_eps_guid * Fundamentals.shares_outstanding)
# fqn2_sales_yield_guid = fqn2_sales_guid/(fqn2_sales_guid - fqn2_eps_guid * Fundamentals.shares_outstanding)
# fqn3_sales_yield_guid = fqn3_sales_guid/(fqn3_sales_guid - fqn3_eps_guid * Fundamentals.shares_outstanding)
# fqn4_sales_yield_guid = fqn4_sales_guid/(fqn4_sales_guid - fqn4_eps_guid * Fundamentals.shares_outstanding)
# fqn5_sales_yield_guid = fqn5_sales_guid/(fqn5_sales_guid - fqn5_eps_guid * Fundamentals.shares_outstanding)
# fqn6_sales_yield_guid = fqn6_sales_guid/(fqn6_sales_guid - fqn6_eps_guid * Fundamentals.shares_outstanding)
# fqn7_sales_yield_guid = fqn7_sales_guid/(fqn7_sales_guid - fqn7_eps_guid * Fundamentals.shares_outstanding)
# fqn8_sales_yield_guid = fqn8_sales_guid/(fqn8_sales_guid - fqn8_eps_guid * Fundamentals.shares_outstanding)

#SALES YIELD - CONSENSUS
# fq0_sales_yield_cons = fq0_sales_cons/(fq0_sales_cons - fq0_eps_cons * Fundamentals.shares_outstanding)
# fqn1_sales_yield_cons = fqn1_sales_cons/(fqn1_sales_cons - fqn1_eps_cons * Fundamentals.shares_outstanding)
# fqn2_sales_yield_cons = fqn2_sales_cons/(fqn2_sales_cons - fqn2_eps_cons * Fundamentals.shares_outstanding)
# fqn3_sales_yield_cons = fqn3_sales_cons/(fqn3_sales_cons - fqn3_eps_cons * Fundamentals.shares_outstanding)
# fqn4_sales_yield_cons = fqn4_sales_cons/(fqn4_sales_cons - fqn4_eps_cons * Fundamentals.shares_outstanding)
# fqn5_sales_yield_cons = fqn5_sales_cons/(fqn5_sales_cons - fqn5_eps_cons * Fundamentals.shares_outstanding)
# fqn6_sales_yield_cons = fqn6_sales_cons/(fqn6_sales_cons - fqn6_eps_cons * Fundamentals.shares_outstanding)
# fqn7_sales_yield_cons = fqn7_sales_cons/(fqn7_sales_cons - fqn7_eps_cons * Fundamentals.shares_outstanding)
# fqn8_sales_yield_cons = fqn8_sales_cons/(fqn8_sales_cons - fqn8_eps_cons * Fundamentals.shares_outstanding)

# cs_sales_yield_mean_q = output[['fq0_sales_yield_cons', 
# 'fqn1_sales_yield_cons',
# 'fqn2_sales_yield_cons',
# 'fqn3_sales_yield_cons',
# 'fqn4_sales_yield_cons',
# 'fqn5_sales_yield_cons',
# 'fqn7_sales_yield_cons',
# 'fqn8_sales_yield_cons']].mean(axis=1)

# cs_sales_yield_std_q = output[['fq0_sales_yield_cons', 
# 'fqn1_sales_yield_cons',
# 'fqn2_sales_yield_cons',
# 'fqn3_sales_yield_cons',
# 'fqn4_sales_yield_cons',
# 'fqn5_sales_yield_cons',
# 'fqn7_sales_yield_cons',
# 'fqn8_sales_yield_cons']].std(axis=1)

# alpha40 = signalize(cs_sales_yield_mean_q/cs_sales_yield_std_q)


#Consistency of EPS Growth from Consensus vs Actual
# act_to_cons_eps_growth_q0 = (fq1_eps_cons/fq0_eps_act) 
# act_to_cons_eps_growth_qn1 = (fq0_eps_cons/fqn1_eps_act)
# act_to_cons_eps_growth_qn2 = (fqn1_eps_cons/fqn2_eps_act)
# act_to_cons_eps_growth_qn3 = (fqn2_eps_cons/fqn3_eps_act)
# act_to_cons_eps_growth_qn4 = (fqn3_eps_cons/fqn4_eps_act)
# act_to_cons_eps_growth_qn5 = (fqn4_eps_cons/fqn5_eps_act)
# act_to_cons_eps_growth_qn6 = (fqn5_eps_cons/fqn6_eps_act)
# act_to_cons_eps_growth_qn7 = (fqn6_eps_cons/fqn7_eps_act)
# act_to_cons_eps_growth_qn8 = (fqn7_eps_cons/fqn8_eps_act)


#Consistency of Cash Flow Growth from Consensus vs Actual - Test it
# act_to_cons_cfps_growth_q0 = (fq1_cfps_cons/fq0_cfps_act) 
# act_to_cons_cfps_growth_qn1 = (fq0_cfps_cons/fqn1_cfps_act)
# act_to_cons_cfps_growth_qn2 = (fqn1_cfps_cons/fqn2_cfps_act)
# act_to_cons_cfps_growth_qn3 = (fqn2_cfps_cons/fqn3_cfps_act)
# act_to_cons_cfps_growth_qn4 = (fqn3_cfps_cons/fqn4_cfps_act)
# act_to_cons_cfps_growth_qn5 = (fqn4_cfps_cons/fqn5_cfps_act)
# act_to_cons_cfps_growth_qn6 = (fqn5_cfps_cons/fqn6_cfps_act)
# act_to_cons_cfps_growth_qn7 = (fqn6_cfps_cons/fqn7_cfps_act)
# act_to_cons_cfps_growth_qn8 = (fqn7_cfps_cons/fqn8_cfps_act)


# cs_act_to_cons_eps_growth_mean = output[['act_to_cons_eps_growth_q0', 
#                                         'act_to_cons_eps_growth_qn1', 
#                                        'act_to_cons_eps_growth_qn2',
#                                         'act_to_cons_eps_growth_qn3',
#                                         'act_to_cons_eps_growth_qn4',
#                                         'act_to_cons_eps_growth_qn5',
#                                         'act_to_cons_eps_growth_qn6',
#                                         'act_to_cons_eps_growth_qn7',
#                                         'act_to_cons_eps_growth_qn8',
#                                        ]].mean(axis=1) 

# cs_act_to_cons_eps_growth_std = output[['act_to_cons_eps_growth_q0', 
#                                         'act_to_cons_eps_growth_qn1', 
#                                         'act_to_cons_eps_growth_qn2',
#                                        'act_to_cons_eps_growth_qn3',
#                                         'act_to_cons_eps_growth_qn4',
#                                         'act_to_cons_eps_growth_qn5',
#                                         'act_to_cons_eps_growth_qn6',
#                                         'act_to_cons_eps_growth_qn7',
#                                         'act_to_cons_eps_growth_qn8',
#                                        ]].std(axis=1) 


# alpha461 = signalize(cs_act_to_cons_eps_growth_mean)
# alpha462 = signalize(-cs_act_to_cons_eps_growth_std)
# alpha463 = signalize(cs_act_to_cons_eps_growth_mean/cs_act_to_cons_eps_growth_std)


# cs_act_to_cons_cfps_growth_mean = output[['act_to_cons_cfps_growth_q0', 
#                                         'act_to_cons_cfps_growth_qn1', 
#                                        'act_to_cons_cfps_growth_qn2',
#                                         'act_to_cons_cfps_growth_qn3',
#                                         'act_to_cons_cfps_growth_qn4',
#                                         'act_to_cons_cfps_growth_qn5',
#                                         'act_to_cons_cfps_growth_qn6',
#                                         'act_to_cons_cfps_growth_qn7',
#                                         'act_to_cons_cfps_growth_qn8',
#                                        ]].mean(axis=1) 

# cs_act_to_cons_cfps_growth_std = output[['act_to_cons_cfps_growth_q0', 
#                                         'act_to_cons_cfps_growth_qn1', 
#                                         'act_to_cons_cfps_growth_qn2',
#                                        'act_to_cons_cfps_growth_qn3',
#                                         'act_to_cons_cfps_growth_qn4',
#                                         'act_to_cons_cfps_growth_qn5',
#                                         'act_to_cons_cfps_growth_qn6',
#                                         'act_to_cons_cfps_growth_qn7',
#                                         'act_to_cons_cfps_growth_qn8',
#                                        ]].std(axis=1) 


# alpha471 = signalize(cs_act_to_cons_cfps_growth_mean)
# alpha472 = signalize(-cs_act_to_cons_cfps_growth_std)
# alpha473 = signalize(cs_act_to_cons_cfps_growth_mean/cs_act_to_cons_cfps_growth_std)

